# Classificação de imagens com openCV e tensorflow
- (RNA)

## Libs

In [18]:
import cv2
import numpy as np
import os 
import zipfile 
from matplotlib import pyplot as plt 
import pandas as pd 
import seaborn as sns 
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.12.0'

## Extração dos dados não estruturados

In [3]:
# Extração dos pixels da imagem
path = '/home/eric/projetos pessoais/computer-vision/Visão Computacional Guia Completo/Datasets/homer_bart_1.zip'
zip_obj = zipfile.ZipFile(path, mode='r')
zip_obj.extractall('./')
zip_obj.close()

In [4]:
diretorio = '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1'

arquivos = [os.path.join(diretorio, f) for f in sorted(os.listdir(diretorio))]

In [5]:
print(arquivos)

['/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/.DS_Store', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart1.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart10.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart100.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart101.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart102.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart103.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart104.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart105.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart106.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart108.bmp', '/home/eric/projetos pessoais/computer-vision/notebooks/homer_bart_1/bart109.bmp', '/home/e

## Pré-processamento

In [6]:
# tratamento de imagens 
largura = 128
altura = 128

In [7]:
# pixels da imagem
imagens = []
# classes homer e bart
classes = []

In [8]:
for imagem_caminho in arquivos:
    #print(imagem_caminho)
    try:

        imagem = cv2.imread(imagem_caminho)
        # altura e largura
        (h, w) = imagem.shape[:2]
    except:
        continue

    imagem = cv2.resize(imagem, (largura, altura))
    imagem = cv2.cvtColor(imagem , cv2.COLOR_BGR2GRAY)
    #plt.imshow(imagem)
    #plt.show()

    # converter de matriz para vetor
    imagem =  imagem.ravel()
    #print(imagem.shape)
    # add na lista de imagens
    imagens.append(imagem)
    # add classes
    nome_imagem = os.path.basename(os.path.normpath(imagem_caminho))
    #print(nome_imagem)
    if nome_imagem.startswith('b'):
        classe = 0
    else:
        classe = 1
    classes.append(classe)
    #print(classe)


In [9]:
# exemplo bart
imagens[0], classes[0]

(array([255, 255, 255, ..., 255, 255, 255], dtype=uint8), 0)

In [10]:
# exemplo homer
imagens[200], classes[200]

(array([255, 255, 255, ..., 255, 255, 255], dtype=uint8), 1)

## Split inicial

In [11]:
X = np.asarray(imagens)
y = np.asarray(classes)

In [12]:
X.shape

(269, 16384)

In [13]:
y.shape

(269,)

In [14]:
np.unique(y, return_counts=True)

(array([0, 1]), array([160, 109]))

## Normalização dos dados

In [15]:
scaler = MinMaxScaler()

In [16]:
scaler.fit(X)
X = scaler.transform(X)

## Divisão treino e teste

In [19]:
X.shape , y.shape

((269, 16384), (269,))

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((215, 16384), (215,), (54, 16384), (54,))

## Modelagem

    - Construção da rede neural

In [23]:
# neurônios
(16384 + 2 ) / 2

8193.0

In [24]:
network1 = tf.keras.models.Sequential()
# camadas
network1.add(tf.keras.layers.Dense(input_shape = (16384,), units=8183, activation='relu') )
# camada oculta
network1.add(tf.keras.layers.Dense(units=8193, activation='relu'))
# camada saída
network1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [25]:
# estrurura da rede
network1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8183)              134078455 
                                                                 
 dense_1 (Dense)             (None, 8193)              67051512  
                                                                 
 dense_2 (Dense)             (None, 1)                 8194      
                                                                 
Total params: 201,138,161
Trainable params: 201,138,161
Non-trainable params: 0
_________________________________________________________________


In [26]:
# treinamento
network1.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# histórico
historico = network1.fit(X_train, y_train, epochs=50)

Epoch 1/50
7/7 [==============================] - 20s 3s/step - loss: 121.1146 - accuracy: 0.5070
Epoch 2/50
7/7 [==============================] - 18s 3s/step - loss: 10.1305 - accuracy: 0.5070
Epoch 3/50
7/7 [==============================] - 18s 3s/step - loss: 1.1423 - accuracy: 0.4512
Epoch 4/50
7/7 [==============================] - 17s 2s/step - loss: 1.0134 - accuracy: 0.5535
Epoch 5/50
7/7 [==============================] - 18s 3s/step - loss: 1.0768 - accuracy: 0.4651
Epoch 6/50
7/7 [==============================] - 18s 3s/step - loss: 0.8920 - accuracy: 0.5442
Epoch 7/50
7/7 [==============================] - 18s 3s/step - loss: 0.6920 - accuracy: 0.4977
Epoch 8/50
7/7 [==============================] - 17s 2s/step - loss: 0.6756 - accuracy: 0.6093
Epoch 9/50
7/7 [==============================] - 19s 3s/step - loss: 0.6486 - accuracy: 0.6047
Epoch 10/50
7/7 [==============================] - 17s 2s/step - loss: 0.6426 - accuracy: 0.6512
Epoch 11/50
7/7 [===================

## Avaliação

## Salvar e carregar a rede neural 

## Prever nova imagem